In [1]:
!nvidia-smi

Wed Feb 17 04:41:02 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                  N/A |
| N/A   39C    P8     6W /  75W |      0MiB /  7611MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

**Prepare**

In [5]:
import numpy as np
from numpy import genfromtxt
from numpy import asarray
import math
import copy
import os
from PIL import Image 
import cv2

patch_size = 64 #input = 64x64
label_size = 128 #output = 128x128

#get RGGB bayer image
def bayer_reverse(img):
    height,width,c = img.shape;
    tmp = np.zeros([height,width]);
    for i in range( height ):
        for j in range( width ):
            if i % 2 == 0 :
                if j % 2 == 0:
                    tmp[i][j] = img[i][j][0];#R
                else:
                    tmp[i][j] = img[i][j][1];#G
            else :
                if j % 2 == 0:
                    tmp[i][j] = img[i][j][1];#G
                else:
                    tmp[i][j] = img[i][j][2];#B

    return tmp;

#split image to prepare the train set
def split(img,name,dir_path):
    height,width,c = img.shape;
    # print(img.shape)
    count = 0;
    for i in range(0,height,30):
        for j in range(0,width,30):
            if( i + label_size < height and j + label_size < width ):
                tmp = np.zeros([label_size,label_size,3]);
                tmp2 = np.zeros([label_size,label_size,3]);
                
                tmp = img[ i : i + label_size, j : j + label_size,:];
                                
                path = os.path.join(dir_path,'label/'+name.split('.')[0] +'_'+str(count)+'.png')                
                im = Image.fromarray(tmp)               
                tmp2[:,:,0] = tmp[:,:,2]
                tmp2[:,:,1] = tmp[:,:,1]                
                tmp2[:,:,2] = tmp[:,:,0]                
                cv2.imwrite(path,tmp2)

                zoom = im.resize((patch_size,patch_size)) 
                zoom2 = np.zeros([patch_size,patch_size,3]);
                # gray =  np.zeros([patch_size,patch_size]);
                
                zoom = np.array(zoom)
                zoom2[:,:,0] = zoom[:,:,2]
                zoom2[:,:,1] = zoom[:,:,1]                
                zoom2[:,:,2] = zoom[:,:,0]
                
                # zoom2 = bayer_reverse(zoom2)
                #gray = gray/255                
                path = os.path.join(dir_path,'patch/'+name.split('.')[0] +'_'+str(count)+'.png')
                # im = Image.fromarray(zoom2)           
                cv2.imwrite(path, zoom2)

                count = count + 1

def main():
    path = 'drive/My Drive/Colab Notebooks/undergraduate_project'
    if not os.path.exists(os.path.join(path,'patch')):
        os.makedirs(os.path.join(path,'patch'))
    
    if not os.path.exists(os.path.join(path,'label')):
        os.makedirs(os.path.join(path,'label'))
    
    dataset_path = os.path.join(path,'origin')
    entries = os.listdir(dataset_path)
    for entry in entries:
        print(entry)
        img_path = dataset_path + '/' + entry
        img = Image.open(img_path)
        img = np.array(img)   
        split(img,entry,path)



if __name__ == '__main__':
    main()


42078.png
92059.png
97017.png
65019.png
56028.png
67079.png
46076.png
95006.png
55075.png
71046.png
45077.png
42044.png
55067.png
61086.png
66075.png
43083.png
61060.png
59078.png
90076.png
80099.png
78019.png
60079.png
66039.png
94079.png
65074.png
65132.png
43070.png
65010.png
68077.png
87065.png
48055.png
54005.png
76002.png
100075.png
105019.png
104022.png
103041.png
105053.png
100080.png
100098.png
106025.png
106020.png


**Training**

In [6]:
import numpy as np
from tensorflow.keras import layers
from keras.preprocessing import image
import tensorflow as tf 
from keras.models import Model,load_model
from keras.utils import to_categorical
import os
import keras
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from keras.layers import Lambda
from keras.optimizers import Adam
from keras import backend as K
import random
from PIL import Image 
from random import shuffle

batch_sz = 16
oti = 'adam'
lr = 0.0001
e_num = 20

#http://ethen8181.github.io/machine-learning/keras/resnet_cam/resnet_cam.html
#https://ithelp.ithome.com.tw/articles/10223034


def main():
  # train_image = np.load('train_image.npy')
  # train_label = np.load('train_label.npy')

  # test_image = np.load('test_img.npy')
  # test_label = np.load('test_lab.npy')

  train_image = []
  train_label = []

  dir_path = 'drive/My Drive/Colab Notebooks/undergraduate_project'
  patch_path = os.path.join(dir_path,'patch')
  entries = os.listdir(patch_path)
  for entry in entries:
    im = image.load_img(patch_path+'/'+entry, target_size = (64, 64))
    img = image.img_to_array(im)    
    # img = img[:,:,0]    
    # img = img[:,:,np.newaxis]           # Modify here!!!
    train_image.append(img)
  train_image= np.stack(train_image)

  print(train_image.shape)# (x,128,128,1)
  # np.save('train_image',train_image)     

  label_path = os.path.join(dir_path,'label')
  entries = os.listdir(label_path)
  for entry in entries:
    im = image.load_img(label_path+'/'+entry, target_size = (128, 128))
    img = image.img_to_array(im)
    train_label.append(img)
  train_label = np.stack(train_label)

  print(train_label.shape)# (x,256,256,3)
  
  # np.save('train_label',train_label)

  index = [i for i in range(train_image.shape[0])]
  shuffle(index)
  train_image = train_image[index,:,:,:];
  train_label = train_label[index,:,:,:];


  inputs = keras.Input(shape=(None,None,3))

  ##Subpixel Construction
  sub_layer_2 = Lambda(lambda x:tf.nn.space_to_depth(x,2)) 
  init = sub_layer_2(inputs=inputs)


  ##Learning Residual (DCNN)
  ####Conv 3x3x64x64 + PReLu
  x = keras.layers.Conv2D(filters = 64, #feature map number
                     kernel_size = 3, 
                     strides = 1,  # 2
                     padding = 'same', 
                     input_shape = (None,None,3))(init)
  
  x = keras.layers.PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None, shared_axes=[1,2])(x)

  ####Residual Block
  for i in range(6):
    Conv1 = keras.layers.Conv2D(filters = 64, #feature map number
                       kernel_size = 3, 
                       strides = 1,  # 2
                       padding = 'same',
                       input_shape = (None,None,64))(x)
    
    PReLu = keras.layers.PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None, shared_axes=[1,2])(Conv1)
    Conv2 = keras.layers.Conv2D(filters = 64, #feature map number
                       kernel_size = 3, 
                       strides = 1,  # 2
                       padding = 'same',
                       input_shape = (None,None,64))(PReLu)
   
    
    x = keras.layers.Add()([Conv2,x])
    x = keras.layers.PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None, shared_axes=[1,2])(x)

  ####Conv 3x3x64x64 + PReLu
  x = keras.layers.Conv2D(filters = 64, #feature map number
                     kernel_size = 3, 
                     strides = 1,  # 2
                     padding = 'same', 
                     input_shape = (None,None,1))(x)
  
  x = keras.layers.PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None, shared_axes=[1,2])(x)
  
  ####Conv 3x3x64x48
  x = keras.layers.Conv2D(filters = 48, #feature map number
                     kernel_size = 3, 
                     strides = 1,  
                     padding = 'same',                      
                     input_shape = (None,None,64))(x)
  
  ###########Learning Residual (DCNN)############
  
  ##Recovery From Subpixel
  sub_layer = Lambda(lambda x:tf.nn.depth_to_space(x,4)) 
  Residual_Output = sub_layer(inputs=x)
  

  ##Initial Prediction
  R = Lambda(lambda x: x[:,:,:,0])(init)
  G = Lambda(lambda x: x[:,:,:,1:3])(init)
  G = Lambda(lambda x: K.mean(x, axis=3))(G)
  B = Lambda(lambda x: x[:,:,:,3])(init)
  # print(init.shape)
  # print(R.shape)
  # print(G.shape)
  # print(B.shape)
  R = Lambda(lambda x: tf.expand_dims(x, -1))(R)
  G = Lambda(lambda x: tf.expand_dims(x, -1))(G)
  B = Lambda(lambda x: tf.expand_dims(x, -1))(B)
  
  #rgb = tf.keras.backend.stack((R, G,B),axis =  3)
  # print(R.shape)
  rg = keras.layers.Concatenate(axis = 3)([R , G])
  rgb = keras.layers.Concatenate(axis = 3)([rg,B])
  # print(rgb.shape)
  Coarse_Output = keras.layers.UpSampling2D(size=(4, 4))(rgb)



  ## + 
  outputs = keras.layers.Add()([Residual_Output,Coarse_Output])
 
  model = keras.Model(inputs=inputs, outputs=outputs, name="JDMSR_model")
  model.summary()
  model.compile(optimizer=Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08), loss = 'mean_squared_error', metrics = ['mse'])
  
  #histories = Histories()
  checkpoint = ModelCheckpoint(os.path.join(dir_path,'model.hdf5'),verbose=1, monitor='val_loss', 
                                save_best_only=True,save_weights_only=True)
  rrp = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1, mode='min', min_lr=0.0000002)

  history = model.fit(train_image, train_label, epochs=e_num, batch_size=batch_sz,verbose=1,
              validation_split = 0.1,callbacks=[checkpoint,rrp],shuffle = True)
  model.save(os.path.join(dir_path,'model.h5'))

if __name__ == '__main__':
  main()


(3528, 64, 64, 3)
(3528, 128, 128, 3)
Model: "JDMSR_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, None, None, 1 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, None, None, 6 6976        lambda[0][0]                     
__________________________________________________________________________________________________
p_re_lu (PReLU)                 (None, None, None, 6 64          conv2d[0][0]                     
__________________________________________________

**Validation**

In [8]:
from keras.models import load_model
from keras.layers import Lambda
from keras.preprocessing import image
import keras
import tensorflow as tf 
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image 
from keras import backend as K
import os
import math
oti = 'adam'

def create_model():
  inputs = keras.Input(shape=(None,None,3))

  ##Subpixel Construction
  sub_layer_2 = Lambda(lambda x:tf.nn.space_to_depth(x,2)) 
  init = sub_layer_2(inputs=inputs)



  ##Learning Residual (DCNN)
  ####Conv 3x3x64x64 + PReLu
  x = keras.layers.Conv2D(filters = 64, #feature map number
                     kernel_size = 3, 
                     strides = 1,  # 2
                     padding = 'same', 
                     input_shape = (None,None,3))(init)
  
  x = keras.layers.PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None, shared_axes=[1,2])(x)

  ####Residual Block
  for i in range(6):
    Conv1 = keras.layers.Conv2D(filters = 64, #feature map number
                       kernel_size = 3, 
                       strides = 1,  # 2
                       padding = 'same',
                       input_shape = (None,None,64))(x)
    
    PReLu = keras.layers.PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None, shared_axes=[1,2])(Conv1)
    Conv2 = keras.layers.Conv2D(filters = 64, #feature map number
                       kernel_size = 3, 
                       strides = 1,  # 2
                       padding = 'same',
                       input_shape = (None,None,64))(PReLu)
   
    
    x = keras.layers.Add()([Conv2,x])
    x = keras.layers.PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None, shared_axes=[1,2])(x)
  ####Conv 3x3x64x64 + PReLu
  x = keras.layers.Conv2D(filters = 64, #feature map number
                     kernel_size = 3, 
                     strides = 1,  # 2
                     padding = 'same', 
                     input_shape = (None,None,1))(x)
  
  x = keras.layers.PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None, shared_axes=[1,2])(x)
  ####Conv 3x3x64x48
  x = keras.layers.Conv2D(filters = 48, #feature map number
                     kernel_size = 3, 
                     strides = 1,  
                     padding = 'same',                      
                     input_shape = (None,None,64))(x)
  
  ###########Learning Residual (DCNN)############

  ##Recovery From Subpixel
  sub_layer = Lambda(lambda x:tf.nn.depth_to_space(x,4)) 
  Residual_Output = sub_layer(inputs=x)
  

  ##Initial Prediction
  R = Lambda(lambda x: x[:,:,:,0])(init)
  G = Lambda(lambda x: x[:,:,:,1:3])(init)
  G = Lambda(lambda x: K.mean(x, axis=3))(G)
  B = Lambda(lambda x: x[:,:,:,3])(init)
  # print(init.shape)
  # print(R.shape)
  # print(G.shape)
  # print(B.shape)
  R = Lambda(lambda x: tf.expand_dims(x, -1))(R)
  G = Lambda(lambda x: tf.expand_dims(x, -1))(G)
  B = Lambda(lambda x: tf.expand_dims(x, -1))(B)
  
  #rgb = tf.keras.backend.stack((R, G,B),axis =  3)
  # print(R.shape)
  rg = keras.layers.Concatenate(axis = 3)([R , G])
  rgb = keras.layers.Concatenate(axis = 3)([rg,B])
  # print(rgb.shape)
  Coarse_Output = keras.layers.UpSampling2D(size=(4, 4))(rgb)

  outputs = keras.layers.Add()([Residual_Output,Coarse_Output])
 
  model = keras.Model(inputs=inputs, outputs=outputs, name="mnist_model")
  return model

dir_path = 'drive/My Drive/Colab Notebooks/undergraduate_project'
model = create_model()
model.load_weights(os.path.join(dir_path,'model.hdf5'))


kodaout = os.path.join(dir_path, 'kodaout')
if not os.path.exists(kodaout):
        os.makedirs(kodaout)

kodain = os.path.join(dir_path, 'koda')
entries = os.listdir(kodain)
for entry in entries:
    # Test Image
    path = kodain+'/'+entry
    test_image = Image.open(path)
    print(test_image.size)
    test_image = test_image.resize((test_image.size[0]//2, test_image.size[1]//2), Image.BILINEAR)
    print(test_image.size)
    test_image = np.array(test_image)
    print(test_image.shape)
    test_image = test_image[np.newaxis,:,:]
    print(test_image.shape)

    out = model.predict(test_image)
    out = out[0]   
    out = image.array_to_img(out)
    path = kodaout+'/'+entry
    out.save(path)


(768, 512)
(384, 256)
(256, 384, 3)
(1, 256, 384, 3)
(768, 512)
(384, 256)
(256, 384, 3)
(1, 256, 384, 3)
(768, 512)
(384, 256)
(256, 384, 3)
(1, 256, 384, 3)
(512, 768)
(256, 384)
(384, 256, 3)
(1, 384, 256, 3)
(768, 512)
(384, 256)
(256, 384, 3)
(1, 256, 384, 3)
(768, 512)
(384, 256)
(256, 384, 3)
(1, 256, 384, 3)
(768, 512)
(384, 256)
(256, 384, 3)
(1, 256, 384, 3)
(768, 512)
(384, 256)
(256, 384, 3)
(1, 256, 384, 3)
(512, 768)
(256, 384)
(384, 256, 3)
(1, 384, 256, 3)
(512, 768)
(256, 384)
(384, 256, 3)
(1, 384, 256, 3)
(768, 512)
(384, 256)
(256, 384, 3)
(1, 256, 384, 3)
(768, 512)
(384, 256)
(256, 384, 3)
(1, 256, 384, 3)
(768, 512)
(384, 256)
(256, 384, 3)
(1, 256, 384, 3)
(768, 512)
(384, 256)
(256, 384, 3)
(1, 256, 384, 3)
(768, 512)
(384, 256)
(256, 384, 3)
(1, 256, 384, 3)
(768, 512)
(384, 256)
(256, 384, 3)
(1, 256, 384, 3)
(512, 768)
(256, 384)
(384, 256, 3)
(1, 384, 256, 3)
(512, 768)
(256, 384)
(384, 256, 3)
(1, 384, 256, 3)
(512, 768)
(256, 384)
(384, 256, 3)
(1, 384, 2

In [9]:
import cv2
import numpy as np
import math
import os
from google.colab.patches import cv2_imshow
from skimage.measure import compare_ssim

def calculate_psnr(original, contrast):
    mse = np.mean((original - contrast) ** 2)
    if mse == 0:
        return 100
    max_value = 255.0
    return 20 * math.log10(max_value / math.sqrt(mse))
    

path = 'drive/My Drive/Colab Notebooks/undergraduate_project' 
input_path = os.path.join(path,'koda')
output_path = os.path.join(path,'kodaout')
entries = os.listdir(input_path)
count = 0
total_psnr = 0.
total_ssim = 0.

for entry in entries:
  img1 = cv2.imread(os.path.join(input_path,entry))
  img2 = cv2.imread(os.path.join(output_path,entry))
  # img1 = cv2.resize(img1, (img2.shape[1], img2.shape[0]), interpolation=cv2.INTER_CUBIC)
  psnr = calculate_psnr(img1,img2)
  print("PSNR-{0}: {1:.10f}dB".format(entry,psnr))
  ssim = compare_ssim(img1, img2, multichannel=True)
  print("SSIM-{0}: {1:.10f}".format(entry,ssim))
  total_psnr += psnr
  total_ssim += ssim
  count += 1
print(count)
# cv2_imshow(img1)
# cv2_imshow(img2)

total_psnr = total_psnr / count
total_ssim = total_ssim / count
print("\n=====================================")
print("Average PSNR:{:.10f}".format(total_psnr))
print("Average SSIM:{:.10f}".format(total_ssim))




PSNR-kodim01.png: 29.4876033846dB
SSIM-kodim01.png: 0.7167695854
PSNR-kodim02.png: 28.6417166007dB


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.


SSIM-kodim02.png: 0.7850120941
PSNR-kodim03.png: 28.3243515302dB
SSIM-kodim03.png: 0.8404275479
PSNR-kodim04.png: 31.2251869056dB
SSIM-kodim04.png: 0.8464220104
PSNR-kodim05.png: 28.9098128818dB
SSIM-kodim05.png: 0.7561124723
PSNR-kodim07.png: 31.0357224670dB
SSIM-kodim07.png: 0.9000563238
PSNR-kodim06.png: 29.4091253460dB
SSIM-kodim06.png: 0.7729614579
PSNR-kodim08.png: 28.7009069817dB
SSIM-kodim08.png: 0.7409519906
PSNR-kodim09.png: 30.5383448061dB
SSIM-kodim09.png: 0.8721981835
PSNR-kodim10.png: 28.0667277049dB
SSIM-kodim10.png: 0.8625777833
PSNR-kodim11.png: 29.6319469953dB
SSIM-kodim11.png: 0.7831349345
PSNR-kodim12.png: 29.0939296405dB
SSIM-kodim12.png: 0.8571261434
PSNR-kodim14.png: 28.6482851561dB
SSIM-kodim14.png: 0.7464715059
PSNR-kodim13.png: 28.8154000003dB
SSIM-kodim13.png: 0.6554379741
PSNR-kodim15.png: 28.2976967629dB
SSIM-kodim15.png: 0.8218277353
PSNR-kodim16.png: 32.6335597900dB
SSIM-kodim16.png: 0.8330166911
PSNR-kodim17.png: 29.8285834212dB
SSIM-kodim17.png: 0.85207